In [1]:
import pickle
from pathlib import Path
import pandas as pd

import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation.metrics import *
from neuralhydrology.evaluation.evaluate import ESDL_start_evaluation
from neuralhydrology.nh_run import start_run, eval_run, finetune, ESDL_eval_run_test_period, ESDL_start_run
from neuralhydrology.utils.nh_results_ensemble import create_results_ensemble
from neuralhydrology.evaluation.metrics import calculate_metrics
import xarray as xr
from neuralhydrology.utils.config import Config

In [8]:
config = Config(Path("../initial_exploration/parallel_grid_search.yml"))
ESDL_start_run(config, gpu=-1)

2024-04-07 20:30:45,914: Logging to /Users/evanrobert/Documents/ESDL_Research/ESDL_LSTM/neuralhydrology/runs/parallel_grid_search_test_0704_203045/output.log initialized.
2024-04-07 20:30:45,917: ### Folder structure created at /Users/evanrobert/Documents/ESDL_Research/ESDL_LSTM/neuralhydrology/runs/parallel_grid_search_test_0704_203045
2024-04-07 20:30:45,917: ### Run configurations for parallel_grid_search_test
2024-04-07 20:30:45,918: experiment_name: parallel_grid_search_test
2024-04-07 20:30:45,921: train_basin_file: ../initial_exploration/exploration_tuler
2024-04-07 20:30:45,921: validation_basin_file: ../initial_exploration/exploration_tuler
2024-04-07 20:30:45,922: test_basin_file: ../initial_exploration/exploration_tuler
2024-04-07 20:30:45,922: train_start_date: 1981-10-01 00:00:00
2024-04-07 20:30:45,923: train_end_date: 1997-09-30 00:00:00
2024-04-07 20:30:45,924: validation_start_date: 1997-10-01 00:00:00
2024-04-07 20:30:45,926: validation_end_date: 2000-09-30 00:00:00
2

KeyboardInterrupt: 

In [6]:
def grid_search():
    hidden_states = [64, 96, 156, 196, 256]
    dropout_rate = [0.0, 0.25, 0.4, 0.5]
    input_seq_lengths = [90, 180, 270, 365]
    #stacked_layers = [1, 2]

    param_combos = []
    for i in hidden_states:
        for j in dropout_rate:
            for k in input_seq_lengths:
                param_combos.append((i, j, k))

    config_path = Path("../initial_exploration/parallel_grid_search.yml")

    param_nse = []
    # parallel portion
    for c in param_combos:
        config = Config(config_path)
        config.update_config({'hidden_size': c[0]})
        config.update_config({'output_dropout': c[1]})
        config.update_config({'seq_length': c[2]})

        nse = ESDL_ensemble(config, gpu=-1)
        param_nse.append((c, nse))
    
    return param_combos, param_nse
    

In [13]:
def ESDL_ensemble(config, num_ensemble_members=5):
    '''Train ensemble for given config file and return ensemble NSE'''

    #train models
    paths = []
    for i in range(num_ensemble_members):
        ESDL_start_run(config, gpu=-1)
        path = config.run_dir
        paths.append(path)
    
    #evaluate models
    for p in paths:
        eval_run(run_dir=p, period="test")
        eval_run(run_dir=p, period="validation")

    ensemble_run = create_results_ensemble(paths, period='test')
    ensemble_nse = ensemble_run['Tuler']['1D']['NSE']
    return ensemble_nse
